<a href="https://www.kaggle.com/code/skshmjn/bert-ner?scriptVersionId=214644289" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
!pip install transformers datasets tokenizers seqeval evaluate peft -q 
!pip install -U accelerate 
!pip install bitsandbytes

In [15]:
!pip install trl

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 8.5 MB/s eta 0:00:00


In [1]:
import datasets 
import numpy as np 
import torch 
import json
from transformers import pipeline
from evaluate import load
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification , DataCollatorWithPadding
from transformers import AutoModelForTokenClassification , AutoTokenizer
from transformers import TrainingArguments, Trainer 
from transformers import  BitsAndBytesConfig
import os
os.environ["WANDB_DISABLED"] = "true"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
conll2003 = datasets.load_dataset("conll2003", trust_remote_code=True)
conll2003

README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [ ]:
conll2003["train"][0]

In [4]:
ner_classes = conll2003["train"].features["ner_tags"].feature.names
ner_classes

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [ ]:
example = conll2003["train"][345]
example['tokens'] ,[ner_classes[i] for i in example['ner_tags']]

In [5]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
def tokenize_and_align_labels(examples, label_all_tokens=True): 

    # Tokenisation
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True) 
    labels = [] 

    # checking all the labels and adding label for word piece 
    for i, label in enumerate(examples["ner_tags"]):
        
        word_ids = tokenized_inputs.word_ids(batch_index=i) 
        previous_word_idx = None 
        label_ids = []
        
        for word_idx in word_ids: 
            if word_idx is None: 
               
                label_ids.append(0)
           
            elif word_idx != previous_word_idx:
               
                label_ids.append(label[word_idx]) 
            else: 
               
                label_ids.append(label[word_idx] if label_all_tokens else 0) 
                 
            previous_word_idx = word_idx 
        labels.append(label_ids) 
    tokenized_inputs["labels"] = labels 
    return tokenized_inputs 

In [ ]:
result = tokenize_and_align_labels(conll2003['train'][345:346])
print(result)

In [ ]:
for token, label in zip(tokenizer.convert_ids_to_tokens(result["input_ids"][0]),result["labels"][0]): 
    print(f"{token:_<40} {label}") 

In [6]:
tokenized_datasets = conll2003.map(tokenize_and_align_labels, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["tokens","pos_tags", "chunk_tags", "ner_tags"])

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets['train'][0:3]

In [72]:
data_collator = DataCollatorForTokenClassification(tokenizer) 
metric = load("seqeval") 
def compute_metrics(eval_preds): 
 
    pred_logits, labels = eval_preds 
    
    pred_logits = np.argmax(pred_logits, axis=2) 
    
    predictions = [ 
        [ner_classes[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100] 
        for prediction, label in zip(pred_logits, labels) 
    ] 
    
    true_labels = [ 
      [ner_classes[l] for (eval_preds, l) in zip(prediction, label) if l != -100] 
       for prediction, label in zip(pred_logits, labels) 
   ] 
    # print('true-->',true_labels[:4])
    # print('predict-->',predictions[:4])

    results = metric.compute(predictions=predictions, references=true_labels) 
    return { 
   "precision": results["overall_precision"], 
   "recall": results["overall_recall"], 
   "f1": results["overall_f1"], 
  "accuracy": results["overall_accuracy"], 
  } 

In [ ]:
args = TrainingArguments( 
"test-ner",
evaluation_strategy = "epoch", 
learning_rate=2e-5, 
per_device_train_batch_size=64, 
per_device_eval_batch_size=64, 
num_train_epochs=5, 
weight_decay=0.01, 
) 

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=9).to(device)

In [ ]:
trainer = Trainer( 
    model, 
    args, 
   train_dataset=tokenized_datasets["train"], 
   eval_dataset=tokenized_datasets["validation"], 
   data_collator=data_collator, 
   tokenizer=tokenizer, 
   compute_metrics=compute_metrics 
) 

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(tokenized_datasets["test"])

In [ ]:
model.save_pretrained("ner_model")
tokenizer.save_pretrained("tokenizer")

In [ ]:
id2label = {
    str(i): label for i,label in enumerate(ner_classes)
}
label2id = {
    label: str(i) for i,label in enumerate(ner_classes)
}

config = json.load(open("ner_model/config.json"))
config["id2label"] = id2label
config["label2id"] = label2id
json.dump(config, open("ner_model/config.json","w"))

In [ ]:
model_fine_tuned = AutoModelForTokenClassification.from_pretrained("ner_model").to(device)
nlp = pipeline("ner", model=model_fine_tuned, tokenizer=tokenizer)

example = "Narendra Damodardas Modi[a] (born 17 September 1950)[b] is an Indian politician who has served as Prime Minister of India since 2014. Modi was the chief minister of Gujarat from 2001 to 2014 and is the member of parliament (MP) for Varanasi. He is a member of the Bharatiya Janata Party (BJP) and of the Rashtriya Swayamsevak Sangh (RSS), a right-wing Hindu nationalist paramilitary volunteer organisation. He is the longest-serving prime minister outside the Indian National Congress."

ner_results = nlp(example)

print(ner_results)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!sudo apt-get install git-lfs

In [ ]:
trainer.push_to_hub()

## Quantisation

In [ ]:
!pip install --upgrade transformers auto-round

In [ ]:
model_fine_tuned = AutoModelForTokenClassification.from_pretrained("skshmjn/test-ner").to(device)
auto_round_fine_tuned = AutoModelForTokenClassification.from_pretrained("skshmjn/test-ner").to(device)
tokenizer = AutoTokenizer.from_pretrained('skshmjn/test-ner')

In [ ]:
from auto_round import AutoRound
bits, group_size, sym = 4, 128, True
autoround = AutoRound(auto_round_fine_tuned, tokenizer, bits=bits, group_size=group_size, batch_size=2, seqlen=512, sym=sym, gradient_accumulate_steps=4, device=device)
autoround.quantize()
auto_round_fine_tuned.save_pretrained("auto_round_fine_tuned")


In [ ]:
auto_round_fine_tuned = AutoModelForTokenClassification.from_pretrained("auto_round_fine_tuned",torch_dtype=torch.float16)
print_size_of_model(auto_round_fine_tuned)

In [ ]:
quantized_model = torch.quantization.quantize_dynamic(
    model_fine_tuned, {torch.nn.Linear}, dtype=torch.qint8,
).to('cpu')

In [ ]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp_delme.p")
    print('Size (KB):', os.path.getsize("temp_delme.p")/1e3)
    os.remove('temp_delme.p')


In [ ]:
fine_tune_trainer = Trainer( 
    model_fine_tuned, 
    args, 
   train_dataset=tokenized_datasets["train"], 
   eval_dataset=tokenized_datasets["validation"], 
   data_collator=data_collator, 
   tokenizer=tokenizer, 
   compute_metrics=compute_metrics 
) 

quntised_trainer = Trainer( 
    quantized_model, 
    args, 
   train_dataset=tokenized_datasets["train"], 
   eval_dataset=tokenized_datasets["validation"], 
   data_collator=data_collator, 
   tokenizer=tokenizer, 
   compute_metrics=compute_metrics , 
) 

autoround_trainer = Trainer( 
    auto_round_fine_tuned, 
    args, 
   train_dataset=tokenized_datasets["train"], 
   eval_dataset=tokenized_datasets["validation"], 
   data_collator=data_collator, 
   tokenizer=tokenizer, 
   compute_metrics=compute_metrics 
) 

In [ ]:
print_size_of_model(model_fine_tuned), fine_tune_trainer.evaluate(tokenized_datasets["test"])

In [ ]:
print_size_of_model(quantized_model), quntised_trainer.evaluate(tokenized_datasets["test"])

In [ ]:
print_size_of_model(auto_round_fine_tuned), autoround_trainer.evaluate(tokenized_datasets["test"])

In [ ]:
print_size_of_model(model_fine_tuned), print_size_of_model(quantized_model), print_size_of_model(auto_round_fine_tuned)

## LORA


## using standard bert again to fine tune and compared result to earlier fine tuning

In [ ]:
from peft import LoraConfig, TaskType
from peft import get_peft_model

lora_config = LoraConfig(
    task_type=TaskType.TOKEN_CLS, r=1, lora_alpha=1, lora_dropout=0.1
)

In [ ]:
# tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")
model = AutoModelForTokenClassification.from_pretrained(
    'bert-base-uncased', num_labels = 9
)

In [ ]:
peft_model = get_peft_model(model, lora_config)

In [ ]:
args = TrainingArguments( 
"lora-ner",
evaluation_strategy = "epoch", 
learning_rate=1e-3, 
per_device_train_batch_size=64, 
per_device_eval_batch_size=64, 
num_train_epochs=5, 
weight_decay=0.01
) 
trainer = Trainer( 
    model, 
    args, 
   train_dataset=tokenized_datasets["train"], 
   eval_dataset=tokenized_datasets["validation"], 
   data_collator=data_collator, 
   tokenizer=tokenizer, 
   compute_metrics=compute_metrics 
) 

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(tokenized_datasets["test"])

In [ ]:
model.save_pretrained("lora_ner_model")
tokenizer.save_pretrained("lora_ner_model")

In [ ]:
id2label = {
    str(i): label for i,label in enumerate(ner_classes)
}
label2id = {
    label: str(i) for i,label in enumerate(ner_classes)
}

config = json.load(open("/kaggle/working/lora_ner_model/adapter_config.json"))
config["id2label"] = id2label
config["label2id"] = label2id
json.dump(config, open("/kaggle/working/lora_ner_model/adapter_config.json","w"))

In [ ]:
print_size_of_model(peft_model), trainer.evaluate(tokenized_datasets["test"])

## Qlora

In [8]:
from peft import LoraConfig, get_peft_model

In [78]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=9, # For binary classification
    quantization_config=bnb_config,
    torch_dtype=torch.float32# Automatically place model on GPU/CPU
)

`low_cpu_mem_usage` was None, now default to True since model is quantized.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [79]:
lora_config = LoraConfig(
    r=8,  # Rank of the LoRA matrices
    lora_alpha=8,  # Scaling factor
    # target_modules=["query", "key", "value"],  # Target attention modules
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS",  # Sequence classification task
)

# Add LoRA adapters to the model
q_lora_model = get_peft_model(model, lora_config)

In [80]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp_delme.p")
    print('Size (KB):', os.path.getsize("temp_delme.p")/1e3)
    os.remove('temp_delme.p')
print_size_of_model(model), print_size_of_model(q_lora_model)

Size (KB): 141211.103
Size (KB): 141229.535


(None, None)

In [86]:
args = TrainingArguments( 
"qlora-ner",
eval_strategy = "epoch", 
learning_rate=1e-3, 
per_device_train_batch_size=64, 
per_device_eval_batch_size=64, 
num_train_epochs=5, 
weight_decay=0.01,
fp16=True,
report_to="none"
# optim = "paged_adamw_32bit"
) 
trainer = Trainer( 
    q_lora_model, 
    args, 
   train_dataset=tokenized_datasets["train"], 
   eval_dataset=tokenized_datasets["validation"], 
   data_collator=data_collator, 
   processing_class=tokenizer, 
   compute_metrics=compute_metrics 
) 



In [87]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.079036,0.866176,0.881195,0.873621,0.975521
2,No log,0.062367,0.904661,0.911847,0.908240,0.981453
3,0.124300,0.057233,0.910213,0.924264,0.917185,0.982649
4,0.124300,0.056499,0.903637,0.928404,0.915853,0.982418
5,0.061500,0.054520,0.913363,0.930529,0.921866,0.983498


TrainOutput(global_step=1100, training_loss=0.08947390166195957, metrics={'train_runtime': 333.7378, 'train_samples_per_second': 210.36, 'train_steps_per_second': 3.296, 'total_flos': 2064573038563740.0, 'train_loss': 0.08947390166195957, 'epoch': 5.0})

In [88]:
trainer.evaluate(tokenized_datasets["test"])

{'eval_loss': 0.08911669999361038,
 'eval_precision': 0.873308446103593,
 'eval_recall': 0.8884405411820555,
 'eval_f1': 0.8808095070008235,
 'eval_accuracy': 0.9756594169366034,
 'eval_runtime': 8.0047,
 'eval_samples_per_second': 431.372,
 'eval_steps_per_second': 6.746,
 'epoch': 5.0}

In [96]:
q_lora_model.save_pretrained("qlora_ner_model")
tokenizer.save_pretrained("qlora_ner_model")

('qlora_ner_model/tokenizer_config.json',
 'qlora_ner_model/special_tokens_map.json',
 'qlora_ner_model/vocab.txt',
 'qlora_ner_model/added_tokens.json',
 'qlora_ner_model/tokenizer.json')

In [98]:
id2label = {
    str(i): label for i,label in enumerate(ner_classes)
}
label2id = {
    label: str(i) for i,label in enumerate(ner_classes)
}

config = json.load(open("/kaggle/working/qlora_ner_model/adapter_config.json"))
config["id2label"] = id2label
config["label2id"] = label2id
json.dump(config, open("/kaggle/working/qlora_ner_model/adapter_config.json","w"))

In [93]:
from huggingface_hub import notebook_login

notebook_login()

In [99]:
trainer.push_to_hub()

events.out.tfevents.1735065257.c65e69f29fce.105.0:   0%|          | 0.00/5.88k [00:00<?, ?B/s]

Upload 33 LFS files:   0%|          | 0/33 [00:00<?, ?it/s]

events.out.tfevents.1735064843.64bd9cde75e7.89.0:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

events.out.tfevents.1735065743.aff3b00fc5e1.89.1:   0%|          | 0.00/6.82k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

...

events.out.tfevents.1735069845.e5da4b7b0f04.89.13:   0%|          | 0.00/7.02k [00:00<?, ?B/s]

events.out.tfevents.1735069903.e5da4b7b0f04.89.14:   0%|          | 0.00/6.72k [00:00<?, ?B/s]

events.out.tfevents.1735069969.e5da4b7b0f04.89.15:   0%|          | 0.00/6.72k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/skshmjn/qlora-ner/commit/a0ba0c61df21245021686174b70e3beed54a254f', commit_message='End of training', commit_description='', oid='a0ba0c61df21245021686174b70e3beed54a254f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/skshmjn/qlora-ner', endpoint='https://huggingface.co', repo_type='model', repo_id='skshmjn/qlora-ner'), pr_revision=None, pr_num=None)